In [28]:
import pandas as pd
import openai
from sklearn.preprocessing import LabelEncoder                                              

In [29]:
grammer_bank = pd.read_csv("./grammer_data.csv", header=1)

In [30]:
grammer_bank.head()

,대분류,소분류,난이도,문장,해석
0,문장 형식,주어+완전자동사 (S+V),하,You win.,네가 이겼어.
1,문장 형식,주어+완전자동사 (S+V),하,Going up?,올라가나요?
2,문장 형식,주어+완전자동사 (S+V),하,Let's see.,어디 보자.
3,문장 형식,주어+완전자동사 (S+V),하,Let's try.,한 번 해봅시다.
4,문장 형식,주어+완전자동사 (S+V),하,Speak out.,거리낌 없이 얘기 해.


In [31]:
dataset={}
for category in grammer_bank['catetory'].unique():
    dataset[category] = grammer_bank[grammer_bank['catetory']==category][['catetory','detail','sentence']]
    encoder = LabelEncoder()
    dataset[category]['detail']=encoder.fit_transform(dataset[category]['detail'].values)

In [33]:
display(encoder.inverse_transform(dataset[category]['detail']))

array(['가능 여부 표현하기', '가능 여부 표현하기', '가능 여부 표현하기', ..., '인물묘사하기', '인물묘사하기',
       '인물묘사하기'], dtype=object)

In [34]:
dataset.keys()

dict_keys(['문장 형식', '문장의 종류', '문장 구성 요소', '명사 (nouns)', '대명사 (referents)', '동사 (verbs)', '시제 (tense)', '조동사 (modals)', '형용사 (adjectives)', '부사 (adverbs)', '전치사 (prepositions)', '접속사 (conjunctions)', '한정사 (determiners)', '부정 (negation)', '도치 강조 생략 삽입', '분사 (participle)', '동명사 (gerunds)', '부정사 (infinitives)', '비교 (comparison)', '관계사 (relatives)', '태 (voice)', '가정법 (subjunctives)', '문장 전환 (transformation)', '빈출 관용 구문', '숙어 (Idioms)', '의사소통 (communicatives)'])

In [36]:
import pandas as pd
import re
import json
import glob, os

os.makedirs('./dataset', exist_ok=True)
english_pattern = re.compile(r'\b[a-zA-Z]+\b')
df_dict={}
for idx, key in enumerate(dataset.keys()):
    labels = [i for i in dataset[key]['detail']]
    texts = [i for i in dataset[key]['문장']]
    df_dict[key] = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
    en_key= key if english_pattern.findall(key)==[] else english_pattern.findall(key)[0]
    df_json = df_dict[key].to_json(f"dataset/{en_key}.jsonl", orient='records', lines=True)

In [49]:
df_dict['부사 (adverbs)']['completion'].nunique()

23

In [8]:
!pip install --upgrade openai

In [50]:
!openai tools fine_tunes.prepare_data -f dataset/sentence_type.jsonl -q

Analyzing...

- Your file contains 1545 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 1 duplicated prompt-completion sets. These are rows: [334]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://bet

In [66]:
import jsonlines

l=[]
with jsonlines.open("dataset/sentence_type_prepared_train.jsonl") as f:
    for line in f.iter():
    	l.append(line)

In [41]:
!openai api fine_tunes.create -t "dataset/sentence_type_prepared_train.jsonl" -v "dataset/sentence_type_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 22  -m ada

Upload progress: 100%|████████████████████| 83.9k/83.9k [00:00<00:00, 39.0Mit/s]
Uploaded file from dataset/adverbs_prepared_train.jsonl: file-KpvgZ7CpjOdRYTPQt2FyCOwq
Upload progress: 100%|████████████████████| 21.3k/21.3k [00:00<00:00, 19.1Mit/s]
Uploaded file from dataset/adverbs_prepared_valid.jsonl: file-D8jG8Z6m3YeEb4WnleNdV4HD
Created fine-tune: ft-ngWnYhR0kLLcLGzoWuhntAXF
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-02-14 21:45:57] Created fine-tune: ft-ngWnYhR0kLLcLGzoWuhntAXF

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-ngWnYhR0kLLcLGzoWuhntAXF



In [45]:
!openai api fine_tunes.follow -i ft-ngWnYhR0kLLcLGzoWuhntAXF

[2023-02-14 21:45:57] Created fine-tune: ft-ngWnYhR0kLLcLGzoWuhntAXF
[2023-02-14 21:50:56] Fine-tune failed. Errors:
The number of classes in file-D8jG8Z6m3YeEb4WnleNdV4HD does not match the number of classes specified in the hyperparameters.

Job failed. Please contact support@openai.com if you need assistance.


In [23]:
if not os.path.exists('./result'):
    os.makedirs('./result')
    print("make result folder done")

In [44]:
!openai api fine_tunes.results -i ft-ngWnYhR0kLLcLGzoWuhntAXF > result/adverbs.csv

Error: No results file available for fine-tune ft-ngWnYhR0kLLcLGzoWuhntAXF


In [43]:
!openai api fine_tunes.follow -i ft-7Eye2bO8HEV1Ycu224FvVMjD

[2023-02-14 19:23:14] Created fine-tune: ft-7Eye2bO8HEV1Ycu224FvVMjD
[2023-02-14 19:31:00] Fine-tune costs $0.11
[2023-02-14 19:31:00] Fine-tune enqueued. Queue number: 2
[2023-02-14 19:31:40] Fine-tune is in the queue. Queue number: 1
[2023-02-14 19:34:43] Fine-tune is in the queue. Queue number: 0
[2023-02-14 19:40:10] Fine-tune started
[2023-02-14 19:40:10] If `compute_classification_metrics` is `True`, each of the classes must start with a different token. You can view your class tokenizations at https://platform.openai.com/tokenizer?view=bpe.. Fine-tune failed. For help, please contact OpenAI and include your fine-tune ID: ft-7Eye2bO8HEV1Ycu224FvVMjD

Job failed. Please contact support@openai.com if you need assistance.


In [ ]:
results = pd.read_csv('result/result.csv')
results[results['classification/accuracy'].notnull()].tail(1)

In [ ]:
results[results['classification/accuracy'].notnull()]['classification/accuracy'].plot()

In [ ]:
test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head()

In [ ]:
ft_model = 'ada:ft-openai-2021-07-30-12-26-20'
res = openai.Completion.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0)
res['choices'][0]['text']